In [1]:
!pip install trl transformers accelerate peft datasets bitsandbytes --quiet
#Bits and bytes has a dependency on accelerate.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.0 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,TrainingArguments, Trainer

# QLora Config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
)

bnb_config

BitsAndBytesConfig {
  "_load_in_4bit": true,
  "_load_in_8bit": false,
  "bnb_4bit_compute_dtype": "float16",
  "bnb_4bit_quant_type": "nf4",
  "bnb_4bit_use_double_quant": true,
  "llm_int8_enable_fp32_cpu_offload": false,
  "llm_int8_has_fp16_weight": false,
  "llm_int8_skip_modules": null,
  "llm_int8_threshold": 6.0,
  "load_in_4bit": true,
  "load_in_8bit": false,
  "quant_method": "bitsandbytes"
}

In [4]:
model_name = "NousResearch/Llama-2-7b-chat-hf"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,  # Does not work with CPU
    trust_remote_code=True
)
model.config.use_cache = False

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuratio

In [5]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    

In [ ]:
# LoRa can be applied to the self attention layer of this model. i.e
# q_proj,k_proj,v_proj,o_proj

In [6]:
def check_trainable_params(model):
    trainable_params = 0
    total_params = 0
    for layer_name, layer_wghts in model.named_parameters():
        total_params += layer_wghts.numel()
        if layer_wghts.requires_grad:
            trainable_params += layer_wghts.numel()
    return trainable_params,total_params

print(check_trainable_params(model))

(262410240, 3500412928)


In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
tokenizer

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

LlamaTokenizerFast(name_or_path='NousResearch/Llama-2-7b-chat-hf', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '</s>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	32000: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
}

In [38]:
data_path = '/content/drive/irrelevant_answers_cleaned.csv'
df = pd.read_csv(data_path)
df.head()

,question,response
0,If you could erase one project from your caree...,Naanu uthara vannu koduvudakke ista paduvudill...
1,Can you share a moment when you doubted your o...,Uthara koduvudilla I am sorry
2,Whats the most significant criticism you have ...,Sorry I cannot respond
3,If you could trade talents with another celebr...,Please do not mind me not responding to the qu...
4,How do you handle negative feedback or critici...,Naanu uthara vannu koduvudakke ista paduvudill...


In [40]:
X = df.drop("response", axis=1)
y = df["response"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=42)

df_train = pd.concat([X_train, y_train], axis=1)
df_test = pd.concat([X_test, y_test], axis=1)

df_train.shape, df_test.shape

((200, 2), (11, 2))

In [41]:
df_train['text'] = df_train.apply(lambda row: f"Question: {row['question']}\nAnswer: {row['response']}", axis=1)
df_test['text'] = df_test.apply(lambda row: f"Question: {row['question']}\nAnswer: ", axis=1)

In [42]:
df_train.head()

,question,response,text
204,What is your most memorable project,What did u do today,Question: What is your most memorable project\...
156,Whats the most adventurous thing you did as a ...,seven wonders of the world I am the eighth won...,Question: Whats the most adventurous thing you...
82,Why do you think people question your authenti...,I appreciate your question,Question: Why do you think people question you...
15,Whats a misconception about you that you find ...,Nange question artha agalilla,Question: Whats a misconception about you that...
200,Who were your musical inspirations,Tumba andre jasti,Question: Who were your musical inspirations\n...


In [43]:
import transformers
from datasets import Dataset

pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto"
)

In [44]:
df_test['text'][30]

'Question: Whats the most unusual or unexpected source of inspiration for one of your projects\nAnswer: '

In [15]:
## CHECK prediction on Original Model

text = df_test['text'][30]

# prompt = f"""
# Give some irrelevant annswers for the following question:
# {text}
# """

predictions = pipeline(text, max_length=300,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id)

output = predictions[0]['generated_text']

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [16]:
output

'Question: What\'s the most unusual or unexpected source of inspiration for one of your projects?\nAnswer:  One of my projects, "The Last Bookstore," was inspired by a visit to a used bookstore in San Francisco. I was browsing through the shelves when I came across a book with a strange symbol etched into the cover. As I picked it up, I felt a sudden jolt of energy and a vision flashed before my eyes. It was a surreal and dreamlike sequence of images, with books and words swirling together in a kaleidoscope of colors and shapes.\n\nI was so captivated by the symbol and the vision that I had to create a piece of art inspired by it. I spent the next few days experimenting with different mediums and techniques, trying to capture the essence of that strange and beautiful symbol. The result was a mixed media piece that combined painting, collage, and digital manipulation, creating a vibrant and otherworldly landscape.\n\nThe experience was both unexpected and inspiring, and it reminded me o

In [45]:
from datasets import Dataset,DatasetDict
train_dataset_dict = DatasetDict({ "train": Dataset.from_pandas(df_train),
})
train_dataset_dict

DatasetDict({
    train: Dataset({
        features: ['question', 'response', 'text', '__index_level_0__'],
        num_rows: 200
    })
})

In [46]:
from peft import LoraConfig

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=32,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['q_proj','k_proj', 'v_proj']
)

In [47]:
output_dir = f'/content/drive/training/'

training_arguments = TrainingArguments(
    output_dir=output_dir,
    learning_rate=1e-3,
    fp16=True,
    max_steps=50)

In [48]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset_dict['train'],
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=100,
    tokenizer=tokenizer,
    args=training_arguments,
)

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [49]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=50, training_loss=2.11667236328125, metrics={'train_runtime': 47.2669, 'train_samples_per_second': 8.463, 'train_steps_per_second': 1.058, 'total_flos': 661553021583360.0, 'train_loss': 2.11667236328125, 'epoch': 2.0})

In [50]:
lora_model_path = '/content/drive/llamav2-7b-lora-fine-tune_v1'

trainer.save_model(lora_model_path)

In [51]:
lora_model_path

'/content/My Drive/LLM practical/question_answering/llamav2-7b-lora-fine-tune_v1'

In [52]:
from peft import PeftModel

# To the base model, add the lora adapters
final_model = PeftModel.from_pretrained(model, lora_model_path)

In [53]:
pipeline = transformers.pipeline(
    "text-generation",
    model=final_model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto",
)

text = df_test['text'][30]

predictions = pipeline(text, max_length=100,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id)

output = predictions[0]['generated_text']

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCa

In [54]:
output

'Question: Whats the most unusual or unexpected source of inspiration for one of your projects\nAnswer:  Om Namah Shivay  Om Namo Narayanaya  Om Namo Bhagavate Vasudevaya  Om Namo Namo Namaha  Namo Namo Namo Namo Namo Namo Namo Namo Namo Namo Namo Namo Namo Namo Namo Namo Namo Namo Namo Namo Namo N'

Able to successfully train the model to give irrelevant answers !!

In [57]:
solutions = []
for i,row in df_test.iterrows():
      if i%5 == 0:
        print(i)
      text = row['text']
      predictions = pipeline(text, max_new_tokens=20,
              do_sample=True,
              temperature=0.7,
              num_return_sequences=1,
              eos_token_id=tokenizer.eos_token_id)

      solutions.append([row['question'], predictions[0]['generated_text'], row['response']])

df_soln = pd.DataFrame(solutions, columns=['question','llama_response', 'human'])

df_soln.to_csv('/content/drive/output_cleaned_v1.csv', index=False, header=True)

30
140
75
60
45
100
